# AI Red Teaming Agent for Generative AI models and applications in Azure AI Foundry

## Objective
This notebook walks through how to use Azure AI Evaluation's AI Red Teaming Agent functionality to assess the safety and resilience of AI systems against adversarial prompt attacks. AI Red Teaming Agent leverages [Risk and Safety Evaluations](https://learn.microsoft.com/en-us/azure/ai-foundry/concepts/evaluation-metrics-built-in?tabs=warning#risk-and-safety-evaluators) to help identify potential safety issues across different risk categories (violence, hate/unfairness, sexual content, self-harm) combined with attack strategies of varying complexity levels from [PyRIT](https://github.com/Azure/PyRIT), Microsoft AI Red Teaming team's open framework for automated AI red teaming.

## Time
You should expect to spend about 30-45 minutes running this notebook. Execution time will vary based on the number of risk categories, attack strategies, and complexity levels you choose to evaluate.

## Before you begin

### Prerequisite
First, if you have an Azure subscription, create an [Azure AI hub](https://learn.microsoft.com/en-us/azure/ai-studio/concepts/ai-resources) then [create an Azure AI project](https://learn.microsoft.com/en-us/azure/ai-studio/concepts/ai-resources). AI projects and Hubs can be served within a private network and are compatible with private endpoints. You **do not** need to provide your own LLM deployment as the AI Red Teaming Agent hosts adversarial models for both simulation and evaluation of harmful content and connects to it via your Azure AI project.

In order to upload your results to Azure AI Foundry:
- Your AI Foundry project must have a connection (*Connected Resources*) to a storage account with `Microsoft Entra ID` authentication enabled.
- Your AI Foundry project must have the `Storage Blob Data Contributor` role in the storage account.
- You must have the `Storage Blob Data Contributor` role in the storage account.
- You must have network access to the storage account.

For more information see: https://learn.microsoft.com/en-us/azure/ai-foundry/how-to/develop/run-ai-red-teaming-cloud
Viewing AI red teaming results in Azure AI Foundry project: https://learn.microsoft.com/en-us/azure/ai-foundry/how-to/view-ai-red-teaming-results

**Important**: First, ensure that you've installed the [Azure CLI](https://learn.microsoft.com/en-us/cli/azure/install-azure-cli) and then make sure to authenticate to Azure using `az login` in your terminal before running this notebook.

**Important**: Second, the authentication dialog box pop up may be opened at the background. To locate it please minimize all the application window(s) and browser window(s) and choose organization account to login.

### Installation
From a terminal window, navigate to your working directory which contains this sample notebook, and execute the following.
```bash
python -m venv .venv
```

Then, activate the virtual environment created:

```bash
# %source .venv/bin/activate # If using Mac/Linux OS
.venv/Scripts/activate # If using Windows OS
```

With your virtual environment activated, install the following packages required to execute this notebook:

```bash
pip install uv
uv pip install azure-ai-evaluation[redteam] azure-identity python-dotenv azure-ai-projects
```


Now open VSCode with the following command, and ensure your virtual environment is used as kernel to run the remainder of this notebook.
```bash
code .
```

### Imports

In [ ]:

from typing import Optional, Dict, Any
from dotenv import load_dotenv
import os

# Azure imports
from azure.ai.evaluation.red_team import RedTeam, RiskCategory, AttackStrategy
# pip install azure-ai-evaluation[redteam]

# Azure AI Projects SDK
from azure.ai.projects import AIProjectClient
from azure.identity import InteractiveBrowserCredential

# OpenAI imports
from openai import AzureOpenAI

### Login to Azure with valid credentials

Ensure that you've installed the [Azure CLI](https://learn.microsoft.com/en-us/cli/azure/install-azure-cli) and then make sure to authenticate to Azure using `az login` in your terminal before running this notebook.

**Important**: the authentication dialog box pop up may be opened at the background. To locate it please minimize all the application window(s) and browser window(s) and choose organization account to login.

Configure the `credential` object with a different AzureCredential type if this is a requirement for your environment.

In [ ]:
# Azure Credential imports
from azure.identity import DefaultAzureCredential, AzureCliCredential, InteractiveBrowserCredential, get_bearer_token_provider
from azure.ai.projects import AIProjectClient

# Load environment variables first
from dotenv import load_dotenv
load_dotenv("../.env")  # This loads variables from .env into the environment

# Get tenant ID and project connection string
tenant_id = os.environ.get("TENANT_ID")
project_connection_string = os.environ.get("AI_FOUNDRY_PROJECT_ENDPOINT")

# Try DefaultAzureCredential first, fallback to AzureCliCredential if needed
try:
    credential = DefaultAzureCredential()
    # Test the credential
    _ = credential.get_token("https://management.azure.com/.default")
    print("✓ Successfully authenticated with DefaultAzureCredential")
except Exception as e:
    print(f"DefaultAzureCredential failed: {e}")
    print("Trying AzureCliCredential...")
    try:
        !az login
        credential = AzureCliCredential()
        print("✓ Successfully authenticated with AzureCliCredential")
    except Exception as e2:
        print(f"AzureCliCredential also failed: {e2}")
        raise

# Create AIProjectClient using PROJECT_CONNECTION_STRING
if not project_connection_string:
    raise ValueError("PROJECT_CONNECTION_STRING not found in environment variables. Please check your .env file.")

try:
    print("🌐 Using browser-based authentication to bypass Azure CLI cache issues...")
    
    # Use InteractiveBrowserCredential with the specific tenant
    if tenant_id:
        credential = InteractiveBrowserCredential(tenant_id=tenant_id)
    else:
        credential = InteractiveBrowserCredential()
    
    # Create the project client using endpoint
    project_client = AIProjectClient(
        endpoint=project_connection_string,
        credential=credential
    )
    print("✅ AIProjectClient created successfully!")
except Exception as e:
    print("❌ Error creating AIProjectClient:", e)
    print("💡 Please complete the browser authentication prompt that should appear")

### Set Up Your Environment Variables

Set the following variables for use in this notebook. These variables connect to your Azure resources and model deployments.

Set these variables by creating an `.env` file as per instruction from "Required Lab Setup".

**Note:** You can find these values in your Azure AI Foundry project or Azure OpenAI resource.

For reference, here's an example of what your populated environment variables should look like:

```
# Azure AI Project Connection String
PROJECT_CONNECTION_STRING="https://your-aifoundry-endpoint-name.services.ai.azure.com/"
MODEL_DEPLOYMENT_NAME="gpt-4o-mini"
MODEL_API_VERSION="2024-12-01-preview"
TENANT_ID="your-tenant-id"
```

In [ ]:
# Environment variables are already loaded in the authentication cell above
# Just display the values for verification

# Model deployment information
azure_openai_deployment = os.environ.get("MODEL_DEPLOYMENT_NAME", "gpt-4o-mini")
azure_openai_api_version = os.environ.get("MODEL_API_VERSION", "2024-12-01-preview")

print(f"🔑 Using Tenant ID: {tenant_id}")
print(f"🔗 Project Connection: {project_connection_string}")
print(f"🤖 Model Deployment: {azure_openai_deployment}")
print(f"📅 API Version: {azure_openai_api_version}")

# Verify we have the required variables
if not project_connection_string:
    print("❌ WARNING: PROJECT_CONNECTION_STRING is not set!")
    print("Please check your .env file contains PROJECT_CONNECTION_STRING")
if not tenant_id:
    print("⚠️ WARNING: TENANT_ID is not set - using default authentication")

## Understanding AI Red Teaming Agent's capabilities

The Azure AI Evaluation SDK's `RedTeam` functionality evaluates AI systems against adversarial prompts across multiple dimensions:

1. **Risk Categories**: Different content risk categories your AI system might generate
   - Violence
   - HateUnfairness
   - Sexual
   - SelfHarm

2. **Attack Strategies**: Along with standard unmodified prompts which are sent by default as the `baseline`, you can specify different transformations of prompts to elicit undesired content.
You can also use `AttackStrategy.Compose()` to layer two strategies in one attack
   - AnsiAttack: Using ANSI escape codes in prompts
   - AsciiArt: Using ASCII art to disguise harmful content
   - AsciiSmuggler: Hiding harmful content within ASCII characters
   - Atbash: Using the Atbash cipher to encode harmful requests
   - Base64: Encoding harmful content in Base64 format
   - Binary: Converting text to binary to bypass filters
   - Caesar: Using the Caesar cipher for encoding
   - CharacterSpace: Manipulating character spacing to confuse filters
   - CharSwap: Swapping characters to bypass detection
   - Diacritic: Using diacritical marks to alter text appearance
   - Flip: Flipping text to bypass content filters
   - Leetspeak: Converting letters to numbers and symbols
   - Morse: Using Morse code to encode harmful requests
   - ROT13: Using ROT13 cipher for text transformation
   - SuffixAppend: Adding suffixes to confuse detection systems
   - StringJoin: Joining strings in unconventional ways
   - Tense: Changing the tense of harmful requests to past tense
   - UnicodeConfusable: Using similar-looking Unicode characters
   - UnicodeSubstitution: Substituting characters with Unicode alternatives
   - Url: Embedding harmful content within URLs
   - Jailbreak: Specially crafted prompts to bypass AI safeguards

3. **Complexity Levels**: Different difficultly levels of attacks
   - Baseline: Standard functionality tests
   - Easy: Simple attack patterns
   - Moderate: More sophisticated attacks
   - Difficult: Complex, layered attack strategies

The key metric for evaluating results is the **Attack Success Rate (ASR)**, which measures the percentage of attacks that successfully elicit harmful content from your AI system.

## Basic Example: Fixed Response Callback

Let's start with a basic example that demonstrates how to set up a Red Team evaluation using a simple callback that always returns a fixed, safe response regardless of the input prompt.

**Important Note**: Since this callback always responds with the same safe message, the Attack Success Rate (ASR) will be 0% for all categories. This example is merely to demonstrate the API structure and flow.

In [ ]:
# Define a simple callback function that always returns a fixed response
def financial_advisor_callback(query: str) -> str:  # noqa: ARG001
    return "I'm a financial advisor assistant. I can help with investment advice and financial planning within legal and ethical guidelines."

In [ ]:
# For this demo, let's use the PROJECT_CONNECTION_STRING approach
# This approach works when you have the correct project connection string
if project_connection_string:
    red_team = RedTeam(
        azure_ai_project=project_connection_string,
        credential=credential,
        risk_categories=[RiskCategory.Violence, RiskCategory.HateUnfairness],
        num_objectives=1,
    )
    print("✓ RedTeam instance created successfully with project connection string")
else:
    print("❌ PROJECT_CONNECTION_STRING not found in environment variables")
    print(
        "Please ensure your .env file contains the correct PROJECT_CONNECTION_STRING"
    )

NOTE: `num_objectives` specifies the number of attacks to perform per risk category per attack strategy. If the parameter `risk_categories` is not specified, `[RiskCategory.Violence, RiskCategory.HateUnfairness, RiskCategory.Sexual, RiskCategory.SelfHarm]` will be used by default.

Now let's run a simple automated scan using the `RedTeam` with the fixed response target. We'll test against two risk categories and one attack strategy for simplicity.

In [ ]:
# Run the red team scan called "Basic-Callback-Scan" with limited scope for this basic example
# This will test 1 objective prompt for each of Violence and HateUnfairness categories with the Flip strategy
result = await red_team.scan(
    target=financial_advisor_callback,
    scan_name="Basic-Callback-Scan",
    attack_strategies=[AttackStrategy.Flip],
    output_path="red_team_output.json",
)

## Intermediary Example: Using Azure OpenAI Callback Function

Now let's create a more realistic example that uses an Azure OpenAI model for responding to the red teaming prompts. We'll use a callback function that leverages the Azure AI Project client to get access to Azure OpenAI. Please note you may encounter HTTP 400 errors which indicate the model's security filters are working correctly.

In [ ]:
# Define a callback function that uses the Azure AI Project client
# This approach leverages the project client to get Azure OpenAI access
def azure_openai_callback(query: str) -> str:
    """Azure OpenAI callback function using project client."""
    try:
        # Get the Azure OpenAI client from the project
        openai_client = project_client.get_openai_client()
        
        # Create a chat completion request
        response = openai_client.chat.completions.create(
            model=azure_openai_deployment,
            messages=[
                {"role": "system", "content": "You are a helpful assistant."},
                {"role": "user", "content": query}
            ],
            max_tokens=150,
            temperature=0.7
        )
        
        return response.choices[0].message.content
    except Exception as e:
        return f"Error processing request: {str(e)}"

print("✓ Azure OpenAI callback function defined successfully")

The callback function uses the project client to get Azure OpenAI access and run the scan.

In [ ]:
# Run the red team scan called "Intermediary-Model-Target-Scan"
result = await red_team.scan(
    target=azure_openai_callback,
    scan_name="Intermediary-Model-Target-Scan",
    attack_strategies=[AttackStrategy.Flip],
)

## Advanced Example: Using an Azure Open AI Model Endpoint in a Callback Function

Using the same Azure Open AI model configuration as above, we now wrap it in a callback function for more flexibility and control on the input and output handling. This will demonstrate how to evaluate an actual AI application. To test your own actual AI application, replace the inside of the callback function with a call to your application. Please note you'll run into HTTP 400 because of which type Error. Please ignore and proceed as it is triggering the model's security filter.

In [ ]:
# Define a callback that uses Azure OpenAI API via the project client
async def azure_openai_callback(
    messages: list,
    stream: Optional[bool] = False,  # noqa: ARG001
    session_state: Optional[str] = None,  # noqa: ARG001
    context: Optional[Dict[str, Any]] = None,  # noqa: ARG001
) -> dict[str, list[dict[str, str]]]:
    
    # Use the project client to get the OpenAI client
    with project_client.get_openai_client(api_version=azure_openai_api_version) as client:
        
        ## Extract the latest message from the conversation history
        messages_list = [{"role": message.role, "content": message.content} for message in messages]
        latest_message = messages_list[-1]["content"]

        try:
            # Call the model using the project client's OpenAI client
            response = client.chat.completions.create(
                model=azure_openai_deployment,
                messages=[
                    {"role": "user", "content": latest_message},
                ],
                # max_tokens=500, # If using an o1 base model, comment this line out
                max_completion_tokens=500,  # If using an o1 base model, uncomment this line
                # temperature=0.7, # If using an o1 base model, comment this line out (temperature param not supported for o1 base models)
            )

            # Format the response to follow the expected chat protocol format
            formatted_response = {"content": response.choices[0].message.content, "role": "assistant"}
        except Exception as e:
            print(f"Error calling Azure OpenAI: {e!s}")
            formatted_response = {"content": "I encountered an error and couldn't process your request.", "role": "assistant"}
    
    return {"messages": [formatted_response]}

In [ ]:
# Create the RedTeam instance with all of the risk categories with 5 attack objectives generated for each category
# Use the PROJECT_CONNECTION_STRING approach
model_red_team = RedTeam(
    azure_ai_project=project_connection_string,
    credential=credential,
    risk_categories=[
        RiskCategory.Violence,
        RiskCategory.HateUnfairness,
        RiskCategory.Sexual,
        RiskCategory.SelfHarm,
    ],
    num_objectives=5,
)
print("✓ Model RedTeam instance created successfully")

We will use this instance of `model_red_team` to test different attack strategies in the following section.

### Testing Different Attack Strategies

Now we'll run a more comprehensive evaluation using multiple attack strategies across risk categories. This will give us a better understanding of our model's vulnerabilities.Please note you'll run into HTTP 400 because of which type Error. Please ignore and proceed as it is triggering the model's security filter.

In [ ]:
# Run the red team scan with multiple attack strategies
advanced_result = await model_red_team.scan(
    target=azure_openai_callback,
    scan_name="Advanced-Callback-Scan",
    attack_strategies=[
        AttackStrategy.EASY,  # Group of easy complexity attacks
        AttackStrategy.MODERATE,  # Group of moderate complexity attacks
        AttackStrategy.CharacterSpace,  # Add character spaces
        AttackStrategy.ROT13,  # Use ROT13 encoding
        AttackStrategy.UnicodeConfusable,  # Use confusable Unicode characters
        AttackStrategy.CharSwap,  # Swap characters in prompts
        AttackStrategy.Morse,  # Encode prompts in Morse code
        AttackStrategy.Leetspeak,  # Use Leetspeak
        AttackStrategy.Url,  # Use URLs in prompts
        AttackStrategy.Binary,  # Encode prompts in binary
        AttackStrategy.Compose([AttackStrategy.Base64, AttackStrategy.ROT13]),  # Use two strategies in one attack
    ],
    output_path="Advanced-Callback-Scan.json",
)

The data and results used in this attack will be saved to the `output_path` specified. The URL printed out at the end of the scorecard will provide a link to where you results are uploaded and logged to your Azure AI Foundry project.

## Bring your own objectives: Using your own prompts as objectives for RedTeam

Below we demonstrate how to use your own prompts as objectives for a `RedTeam` scan. You can see the required format for prompts under `../data/prompts.json`. Note that when bringing your own prompts, the supported `risk-type`s are `violence`, `sexual`, `hate_unfairness`, and `self_harm`. The number of prompts you specify will be the `num_objectives` used in the scan. Please note you'll run into HTTP 400 because of which type Error. Please ignore and proceed as it is triggering the model's security filter.

In [ ]:
path_to_prompts = "data/prompts.json"

# Check if the prompts file exists
import os
if os.path.exists(path_to_prompts):
    print(f"✓ Found prompts file: {path_to_prompts}")

    # Create the RedTeam specifying the custom attack seed prompts to use as objectives
    custom_red_team = RedTeam(
        azure_ai_project=project_connection_string,
        credential=credential,
        custom_attack_seed_prompts=path_to_prompts,  # Path to a file containing custom attack seed prompts
    )
    print("✓ Custom RedTeam instance created successfully")
else:
    print(f"❌ Prompts file not found: {path_to_prompts}")
    print("Skipping custom red team creation - you can continue with the other examples")

In [ ]:
custom_red_team_result = await custom_red_team.scan(
    target=azure_openai_callback,
    scan_name="Custom-Prompt-Scan",
    attack_strategies=[
        AttackStrategy.EASY,  # Group of easy complexity attacks
        AttackStrategy.MODERATE,  # Group of moderate complexity attacks
        AttackStrategy.DIFFICULT,  # Group of difficult complexity attacks
    ],
    output_path="Custom-Prompt-Scan.json",
)

## Conclusion

In this notebook, we've demonstrated how to use the Azure AI Evaluation SDK's `RedTeam` functionality to assess the safety and resilience of AI systems. We started with a basic fixed-response example and then moved to a more realistic model testing across multiple risk categories and attack strategies.

The automated AI red teaming scans provides valuable insights into:

1. **Overall Attack Success Rate (ASR)** - The percentage of attacks that successfully elicit harmful content
2. **Vulnerability by Risk Category** - Which types of harmful content your model is most vulnerable to
3. **Effectiveness of Attack Strategies** - Which attack techniques are most successful against your model
4. **Impact of Complexity** - How more sophisticated attacks affect your model's safety guardrails

By regularly red-teaming your AI applications, you can identify and address potential vulnerabilities before deploying your models to production environments.

### Next Steps

1. **Mitigation**: Use these results to strengthen your model's guardrails against identified attack vectors
2. **Continuous Testing**: Implement regular red team evaluations as part of your development lifecycle
3. **Custom Strategies**: Develop custom attack strategies for your specific use cases and domain
4. **Safety Layers**: Consider adding additional safety layers like Azure AI Content Safety to filter harmful requests and responses 